# Quadratic under translation

Ian recently sent me new notes. Because we are not taking the norm over all variables, the point $x^*$ cannot be chosen as the min-norm point in the set $\{x:Ax=b\}$. Rather, we must choose $x^*$ to be a solution to the following optimization problem:

\begin{align}


In [1]:
function min_norm(A,b,Q=nothing)
    """ Find the point x* in the set Ax=b which is
    closest to the origin in the 2-norm sense.

    If optional diagonal Q is specified, partition problem 
    into x1 and x2, where x1 corresponds to zeros in Q, and
    x2 has remaining variables.
    """
    m,n = size(A)
    if m < n
        x_star = \(A,b)
    else
        println("need m < n")
        return nothing
    end
    if Q != nothing
        p2 = find(diag(Q))
        p1 = setdiff(1:size(Q,1),p2)
        A1 = A[:,p1]
        # A2*A2' is not invertible if A2 is tall.
        # In this case, use only enough rows to
        # make A2 full rank.
        A2 = A[:,p2]
        r = rank(A2)
        A2inv= A2[end-r+1:end,:]
        x1_star = x_star[p1]
        
        x2_star = A2inv'*((A2inv*A2inv')\((b-A1*x1_star)[end-r+1:end]))
        
        return [x1_star,x2_star]#,A1,A2
    else
        return x_star
    end
end

min_norm (generic function with 2 methods)

In [185]:
A = rand(4,5)
b = [1,1,1,1]
Q = diagm([0,0,0,1,1])
x1,x2,A1,A2 = min_norm(A,b,Q)
x = [x1,x2]
A*x

4-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0

In [186]:
A2*x2

4-element Array{Float64,1}:
  0.90582 
 -0.235338
  0.506393
  1.15318 

In [187]:
b - A1*x1

4-element Array{Float64,1}:
  0.90582 
 -0.235338
  0.506393
  1.15318 

In [128]:
(b-A1*x)[end-r+1:end]

LoadError: DimensionMismatch("")
while loading In[128], in expression starting on line 1

In [40]:
A = [0 0 1 0;
    0 0 0 1;
    0 0 0 0]
b = [1,1,1]
x = min_norm(A,b,2)

LoadError: DimensionMismatch("matrix is not square")
while loading In[40], in expression starting on line 5

In [124]:
x

5-element Array{Float64,1}:
   -3.41968 
    0.295269
    2.8044  
 -374.682   
  919.724   

In [121]:
(A2*A2')^-1

2x2 Array{Float64,2}:
  1727.64  -4369.45
 -4369.45  11057.5 

In [104]:
p1 = find(diag(Q))
p2 = setdiff(1:size(Q,1),p1)

2-element Array{Int64,1}:
 3
 4

In [105]:
A1 = A[:,p1]
A2 = A[:,p2]

4x2 Array{Float64,2}:
 0.653251   0.64195  
 0.0959413  0.501246 
 0.739873   0.0973598
 0.0910069  0.786732 

In [109]:
r = rank(A2)
A2[end-r+1:end,:]

2

In [89]:
A[:,3]

3-element Array{Float64,1}:
 0.511124
 0.933266
 0.206815

In [81]:
A2

-155.2992375676046

In [77]:
A*x

LoadError: DimensionMismatch("")
while loading In[77], in expression starting on line 1

In [42]:
?\

Base.\(A, B)

   Matrix division using a polyalgorithm. For input matrices "A" and
   "B", the result "X" is such that "A*X == B" when "A" is
   square.  The solver that is used depends upon the structure of
   "A".  A direct solver is used for upper- or lower triangular
   "A".  For Hermitian "A" (equivalent to symmetric "A" for non-
   complex "A") the "BunchKaufman" factorization is used.
   Otherwise an LU factorization is used. For rectangular "A" the
   result is the minimum-norm least squares solution computed by a
   pivoted QR factorization of "A" and a rank estimate of A based on
   the R factor. For sparse, square "A" the LU factorization (from
   UMFPACK) is used.

Base.\(x, y)

   Left division operator: multiplication of "y" by the inverse of
   "x" on the left. Gives floating-point results for integer
   arguments.


In [35]:
A

3x4 Array{Int64,2}:
 0  0  1  0
 0  0  0  1
 0  0  0  0

In [36]:
b

3-element Array{Int64,1}:
 1
 1
 1

In [34]:
\(A,b)

LoadError: DimensionMismatch("matrix is not square")
while loading In[34], in expression starting on line 1

The purpose of this new method is to find $x^*:Ax^*=b$ so that translation to $z = x - x^*$ works out as

\begin{align}
x^\top Q_\theta x &= x_2^\top x_2 \\
&= (z_2+x_2^*)^\top